# SF-DAT-21 | Unit Project 3

In this project, you will perform a logistic regression on the admissions data we've been working with in Unit Projects 1 and 2.

In [60]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pylab as pl
from sklearn import linear_model
%matplotlib inline

In [4]:
df_raw = pd.read_csv("../../dataset/admissions.csv")
df = df_raw.dropna()
df.head()

,admit,gre,gpa,prestige
0,0,380,3.61,3
1,1,660,3.67,3
2,1,800,4.00,1
3,1,640,3.19,4
4,0,520,2.93,4


## Part 1. Frequency Tables

#### Question 1. Let's create a frequency table of our variables.

In [35]:
# frequency table for prestige and whether or not someone was admitted

df.groupby('admit')['prestige'].value_counts()

admit  prestige
0      2           95
       3           93
       4           55
       1           28
1      2           53
       1           33
       3           28
       4           12
dtype: int64

In [38]:
pd.crosstab(df.admit, df.prestige)

prestige,1.0,2.0,3.0,4.0
admit,,,,
0,28,95,93,55
1,33,53,28,12


## Part 2. Return of dummy variables

#### Question 2.1. Create class or dummy variables for prestige.

In [29]:
dummy_ranks = pd.get_dummies(df.prestige, prefix = 'prestige')

#### Question 2.2. When modeling our class variables, how many do we need?

4 -- one for each value of `prestige`

## Part 3. Hand calculating odds ratios

Develop your intuition about expected outcomes by hand calculating odds ratios.

In [30]:
cols_to_keep = ['admit', 'gre', 'gpa']
handCalc = df[cols_to_keep].join(dummy_ranks.ix[:, 'prestige_1':])
print handCalc.head()

   admit  gre   gpa  prestige_1.0  prestige_2.0  prestige_3.0  prestige_4.0
0      0  380  3.61             0             0             1             0
1      1  660  3.67             0             0             1             0
2      1  800  4.00             1             0             0             0
3      1  640  3.19             0             0             0             1
4      0  520  2.93             0             0             0             1


In [42]:
# crosstab prestige 1 admission
# frequency table cutting prestige and whether or not someone was admitted
pd.crosstab(handCalc['prestige_1.0'], handCalc.admit)

admit,0,1
prestige_1.0,,
0,243,93
1,28,33


#### Question 3.1. Use the cross tab above to calculate the odds of being admitted to grad school if you attended a #1 ranked college.

In [47]:
# I'm interepreting this as "prestige 1.0 is a #1 ranked college.  admission to grad school is defined as admit == 1"
prob_1 = 33.0 / (33.0+28.0) #54.1%
odds_1 = 33.0 / ((33.0+28.0) - 33.0)

print prob_1, odds_1

0.540983606557 1.17857142857


#### Question 3.2. Now calculate the odds of admission if you did not attend a #1 ranked college.

In [48]:
prob_not_1 = 93.0 / (93.0+243.0)   #27.7%
odds_not_1 = 93.0 / (93.0+243.0 - 93.0)   
print prob_not_1, odds_not_1

0.276785714286 0.382716049383


#### Question 3.3. Calculate the odds ratio.

In [49]:
odds_1 / odds_not_1

3.079493087557604

#### Question 3.4. Write this finding in a sentenance:

Answer: Odds ratio greater than 1 implies that going to a #1 ranked college is associated with admission.

#### Question 3.5. Print the cross tab for prestige_4.

In [52]:
pd.crosstab(handCalc['prestige_4.0'], handCalc.admit)

admit,0,1
prestige_4.0,,
0,216,114
1,55,12


#### Question 3.6. Calculate the OR.

In [53]:
prob_4 = 12. / (12. + 55.) 
odds_4 = 12. / (12. + 55. - 12.) 

prob_not_4 = 114. / (114. + 216.)
odds_not_4 = 114. / (114. + 216. - 114.)

odds_ratio_4 = odds_4 / odds_not_4
odds_ratio_4

0.4133971291866028

#### Question 3.7. Write this finding in a sentence.

Answer: odds ratio less than 1 implies that prestige level 4 is not associated/more likely to get into grad school/admission

## Part 4. Analysis

In [73]:
# create a clean data frame for the regression
cols_to_keep = ['admit', 'gre', 'gpa']
data = df[cols_to_keep].join(dummy_ranks.ix[:, 'prestige_2.0':])
print data.head()

   admit  gre   gpa  prestige_2.0  prestige_3.0  prestige_4.0
0      0  380  3.61             0             1             0
1      1  660  3.67             0             1             0
2      1  800  4.00             0             0             0
3      1  640  3.19             0             0             1
4      0  520  2.93             0             0             1


We're going to add a constant term for our Logistic Regression.  The statsmodels function we're going to be using requires that intercepts/constants are specified explicitly.

In [74]:
# manually add the intercept
data['intercept'] = 1.0

#### Question 4.1. Set the covariates to a variable called train_cols.

In [81]:
#train_cols = data[ ['admit', 'gre', 'gpa']]  #original version
train_cols = ['gre', 'gpa', 'prestige_2.0']

def X_y(df):
    X = df[ train_cols]
    y = df['admit']
    return X, y

train_data = data.sample(frac = .6, random_state = 0)
test_data = data.drop(train_data.index)

train_X, train_y = X_y(train_data)
test_X, test_y = X_y(test_data)

#### Question 4.2. Fit the model.

In [82]:
model_prestige2 = linear_model.LogisticRegression()
model_prestige2.fit(train_X, train_y)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

#### Question 4.3. Print the summary results.

In [85]:
print 'intercept    =', model_prestige2.intercept_
print 'coefficients =', model_prestige2.coef_

intercept    = [-2.16610361]
coefficients = [[ 0.00263522 -0.01342274  0.14164604]]


#### Question 4.4. Calculate the odds ratios of the coeffincients and their 95% CI intervals

hint 1: np.exp(X)

hint 2: conf['OR'] = params

        conf.columns = ['2.5%', '97.5%', 'OR']

In [ ]:
# to get odds, use exponential function
#np.exp(X)

#### Question 4.5. Interpret the OR of Prestige_2.

Answer: If OR < 1, prestige 2 does not improve likelihood of deisred outcome.

#### Question 4.6. Interpret the OR of GPA.

Answer:

## Part 5: Predicted probablities


As a way of evaluating our classifier, we're going to recreate the dataset with every logical combination of input values.  This will allow us to see how the predicted probability of admission increases/decreases across different variables.  First we're going to generate the combinations using a helper function called cartesian (above).

We're going to use np.linspace to create a range of values for "gre" and "gpa".  This creates a range of linearly spaced values from a specified min and maximum value--in our case just the min/max observed values.

In [ ]:
def cartesian(arrays, out=None):
    """
    Generate a cartesian product of input arrays.
    Parameters
    ----------
    arrays : list of array-like
        1-D arrays to form the cartesian product of.
    out : ndarray
        Array to place the cartesian product in.
    Returns
    -------
    out : ndarray
        2-D array of shape (M, len(arrays)) containing cartesian products
        formed of input arrays.
    Examples
    --------
    >>> cartesian(([1, 2, 3], [4, 5], [6, 7]))
    array([[1, 4, 6],
           [1, 4, 7],
           [1, 5, 6],
           [1, 5, 7],
           [2, 4, 6],
           [2, 4, 7],
           [2, 5, 6],
           [2, 5, 7],
           [3, 4, 6],
           [3, 4, 7],
           [3, 5, 6],
           [3, 5, 7]])
    """

    arrays = [np.asarray(x) for x in arrays]
    dtype = arrays[0].dtype

    n = np.prod([x.size for x in arrays])
    if out is None:
        out = np.zeros([n, len(arrays)], dtype=dtype)

    m = n / arrays[0].size
    out[:,0] = np.repeat(arrays[0], m)
    if arrays[1:]:
        cartesian(arrays[1:], out=out[0:m,1:])
        for j in xrange(1, arrays[0].size):
            out[j*m:(j+1)*m,1:] = out[0:m,1:]
    return out

In [ ]:
# instead of generating all possible values of GRE and GPA, we're going
# to use an evenly spaced range of 10 values from the min to the max
gres = np.linspace(data['gre'].min(), data['gre'].max(), 10)

print gres
# array([ 220.        ,  284.44444444,  348.88888889,  413.33333333,
#         477.77777778,  542.22222222,  606.66666667,  671.11111111,
#         735.55555556,  800.        ])

gpas = np.linspace(data['gpa'].min(), data['gpa'].max(), 10)

print gpas
# array([ 2.26      ,  2.45333333,  2.64666667,  2.84      ,  3.03333333,
#         3.22666667,  3.42      ,  3.61333333,  3.80666667,  4.        ])

# enumerate all possibilities
combos = pd.DataFrame(cartesian([gres, gpas, [1, 2, 3, 4], [1.]]))

#### Question 5.1. Recreate the dummy variables.

In [ ]:
# recreate the dummy variables
dummy_ranks = pd.get_dummies(df.prestige, prefix = 'prestige')

# keep only what we need for making predictions
# create a clean data frame for the regression
cols_to_keep = ['admit', 'gre', 'gpa']
data = df[cols_to_keep].join(dummy_ranks.ix[:, 'prestige_2.0':])
print data.head()

#### Question 5.2. Make predictions on the enumerated dataset.

#### Question 5.3. Interpret findings for the last 4 observations.

Answer:

## Bonus

Plot the probability of being admitted into graduate school, stratified by GPA and GRE score.

In [ ]:
df.plot(x = probs)